In [1]:
# EDA inicial — Focos de Queimadas (2019–2024)
# Este notebook carrega os CSVs em data/raw/, consolida e gera análises e gráficos básicos.

import sys
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configs
pd.set_option("display.max_columns", 100)
sns.set(style="whitegrid", context="notebook")

PROJECT_ROOT = Path(__file__).resolve().parents[1]
RAW_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJECT_ROOT)
print("Raw dir:", RAW_DIR)
print("Processed dir:", PROCESSED_DIR)


NameError: name '__file__' is not defined

In [ ]:
# Listar CSVs disponíveis e carregar
csv_files = sorted(RAW_DIR.glob("*.csv"))
if not csv_files:
    raise FileNotFoundError(f"Nenhum CSV encontrado em {RAW_DIR}. Coloque arquivos .csv no diretório e reexecute.")

csv_files


In [ ]:
# Carregar e concatenar
usecols = None  # ou liste colunas específicas para otimizar

frames = []
for fp in csv_files:
    try:
        df = pd.read_csv(fp, encoding="utf-8", sep=",", usecols=usecols, low_memory=False)
    except UnicodeDecodeError:
        df = pd.read_csv(fp, encoding="latin1", sep=",", usecols=usecols, low_memory=False)
    df["_source_file"] = fp.name
    frames.append(df)

raw = pd.concat(frames, ignore_index=True)
raw.shape


In [ ]:
# Limpeza mínima: normalizar nomes e datas
# Tentativa de detectar colunas padrões: data, bioma, estado/UF, municipio
cols = {c.lower(): c for c in raw.columns}

def get_col(*cands):
    for c in cands:
        if c in cols:
            return cols[c]
    return None

col_date = get_col("data_pas", "data", "dt", "datetime")
col_bioma = get_col("bioma")
col_uf = get_col("estado", "uf")
col_muni = get_col("municipio", "município", "munic")

if col_date is None:
    raise KeyError("Não encontrei coluna de data (ex.: data_pas). Atualize o mapeamento.")

raw["date"] = pd.to_datetime(raw[col_date], errors="coerce")
if raw["date"].isna().all():
    raise ValueError("Falha ao converter datas. Verifique o formato em data_pas.")

# Normalização simples de texto
for c in [col_bioma, col_uf, col_muni]:
    if c and c in raw.columns:
        raw[c] = raw[c].astype(str).str.strip()

raw["year"] = raw["date"].dt.year
raw["month"] = raw["date"].dt.to_period("M").astype(str)
raw["day"] = raw["date"].dt.date

raw.head()


In [ ]:
# Sumários básicos
print("Linhas:", len(raw))
print("Colunas:", raw.shape[1])
print("Período:", raw['date'].min(), "->", raw['date'].max())

# Contagem diária total
by_day = raw.groupby('day').size().rename('focos').reset_index()
by_day.head(), by_day.tail()


In [ ]:
# Gráficos: série temporal diária (total)
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(by_day['day'], by_day['focos'], color='tab:red', linewidth=1)
ax.set_title('Focos por dia (total)')
ax.set_xlabel('Dia')
ax.set_ylabel('Focos')
plt.tight_layout()
plt.show()


In [ ]:
# Agregações por bioma e UF (se existirem)
plots = []
if col_bioma and col_bioma in raw.columns:
    by_bioma = raw.groupby([col_bioma, 'month']).size().rename('focos').reset_index()
    pivot_bioma = by_bioma.pivot(index='month', columns=col_bioma, values='focos').fillna(0)
    plots.append(('Focos por mês e bioma', pivot_bioma))

if col_uf and col_uf in raw.columns:
    by_uf = raw.groupby([col_uf, 'month']).size().rename('focos').reset_index()
    pivot_uf = by_uf.pivot(index='month', columns=col_uf, values='focos').fillna(0)
    plots.append(('Focos por mês e UF', pivot_uf))

for title, pv in plots:
    fig, ax = plt.subplots(figsize=(12, 5))
    pv.plot(ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Mês')
    ax.set_ylabel('Focos')
    plt.tight_layout()
    plt.show()

len(plots)


In [ ]:
# Exportar artefatos mínimos
by_day.to_csv(PROCESSED_DIR / "focos_por_dia.csv", index=False)
if 'pivot_bioma' in globals():
    pivot_bioma.to_csv(PROCESSED_DIR / "focos_mes_bioma.csv")
if 'pivot_uf' in globals():
    pivot_uf.to_csv(PROCESSED_DIR / "focos_mes_uf.csv")
print("Arquivos exportados em:", PROCESSED_DIR)
